# Parte 5

En el area de la salud, diagnosticar a una persona de una enfermedad de forma rapida y correcta puede llegar
a salvarle la vida. Los encargados de realizar estos diagnosticos, son medicos que, observando examenes y
ciertos indicadores, pueden concluir que enfermedad presenta el paciente. Si el medico se llegase a equivocar,
aparte de que el paciente pueda perder la vida, el medico podra ser demandado por negligencia arriesgando
a~nos de carcel o pagar sumas de dinero considerable, es por estas razones que es importante no cometer
errores.
Pongamonos en el contexto de que usted es contratado para generar un modelo que prediga si es que un
paciente presenta una enfermedad cardiaca a partir de ciertos indicadores, tales como la edad, sexo, presion
6
sangunea, nivel de glicemia, etc. Los datos para trabajar junto a su documentacion pueden ser descargados
ejecutando los siguientes comandos en un terminal (sistemas UNIX)

In [53]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

headers = ['age','sex','chest_pain','blood_p','serum',
'blood_s','electro','max_heart','angina','oldpeak','slope','vessel','thal','normal']
df = pd.read_csv('heart.dat', header=None, names=headers, sep=' ')
print(df.shape)
df.info()
df.describe()
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)


(270, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 14 columns):
age           270 non-null float64
sex           270 non-null float64
chest_pain    270 non-null float64
blood_p       270 non-null float64
serum         270 non-null float64
blood_s       270 non-null float64
electro       270 non-null float64
max_heart     270 non-null float64
angina        270 non-null float64
oldpeak       270 non-null float64
slope         270 non-null float64
vessel        270 non-null float64
thal          270 non-null float64
normal        270 non-null int64
dtypes: float64(13), int64(1)
memory usage: 29.6 KB


In [55]:
from sklearn.preprocessing import StandardScaler
import sklearn.linear_model as lm

X = df_scaled.iloc[:,1:]
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))
y = df_scaled['max_heart']
mascara = np.zeros(len(X))
limit = int(len(X)*0.7)
mascara[:limit] = 1
istrain = mascara== 1
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]

print(len(Xtrain),len(ytrain))
print(len(Xtest),len(ytest))


(189, 189)
(81, 81)


# b) Entrenar y umbral

In [93]:
import numpy.linalg as LA
linreg = lm.LinearRegression(fit_intercept = False)
modelo = linreg.fit(Xtrain, ytrain)
yhat_val = modelo.predict(Xtest)
error_test = np.mean(np.power(yhat_val - ytest, 2))
print "Error de test:",error_test 

from sklearn.metrics import accuracy_score
condicionamiento_train = LA.cond(Xtrain)
condicionamiento_test = LA.cond(Xtest)
print "Condicionamiento tr:",condicionamiento_train
print "Condicionamiento test:",condicionamiento_test

ytest_arr = np.array(ytest)
print "caca"
residuos = []
for i in range(0,81):
    residuos.append(yhat_val[i]- ytest_arr[i])  #Valor predecido - valor real

    
promedio_dif = np.mean(residuos)
desviacion_dif = np.std(residuos)
print "Promedio:",promedio_dif
print "desviacion:",desviacion_dif

enfermos_pred = np.ones(len(ytest_arr))
enfermos_real = np.ones(len(ytest))


#Y_test

promedio = np.mean(ytest)
maximo = np.max(ytest)
minimo = np.min(ytest)
std = np.std(ytest)

print promedio,maximo,minimo,std

for i in range(len(ytest)):
    if(ytest_arr[i] > promedio + 2*std or ytest_arr[i] < promedio - 2*std):
        enfermos_real[i] = 1
    else:
        enfermos_real[i] = 0
        
# Y_predecido
for i in range(len(ytest_arr)):
    if(residuos[i] > promedio_dif + 2*desviacion_dif):
        enfermos_pred[i] = 1
    else:
        enfermos_pred[i] = 0
        
print "Score:", (accuracy_score(enfermos_pred,enfermos_real))

Error de test: 1.07725360801e-30
Condicionamiento tr: 3.22766700604
Condicionamiento test: 4.03700168753
caca
Promedio: 4.81053811326e-16
desviacion: 9.19696057736e-16
0.158627190609 2.26280005561 -2.27817690882 0.87582916395
Score: 0.925925925926
